In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from advertorch.attacks import LinfPGDAttack

torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [2]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [3]:
n_epochs = 15
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 150

random_seed = 1
# torch.backends.cudnn.enabled = True
torch.manual_seed(random_seed)

### Define data loaders and data preprocessing steps

In [4]:
data_preprocess = torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.1307,), (0.3081,))])
# the mean of mnist pixel data is .1307 and the stddev is .3081
# the first,second argument to normalize is a sequence of means,standard devs for each input channel
# Normalize subtracts the mean from each value in each channel and divides by the stddev for the channel


train_loader = torch.utils.data.DataLoader(
                    torchvision.datasets.MNIST('./data', train=True, download=True,
                         transform=data_preprocess), 
                    batch_size=batch_size_train, 
                    shuffle=True)

test_loader = torch.utils.data.DataLoader(
                    torchvision.datasets.MNIST('./data', train=False, download=True,
                         transform=data_preprocess), 
                    batch_size=batch_size_test, 
                    shuffle=True)

### Define the model

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.fc1 = nn.Linear((28-(5-1))*(28-(5-1))*10, 128)
        self.fc2 = nn.Linear(128, 10)
        self.fc1_drop = nn.Dropout(.50)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(-1, (28-4)*(28-4)*10)
        x = F.relu(self.fc1_drop(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Define my loss function for weighting adversarial examples

In [6]:
def my_loss(og_output, og_labels, adv_output, adv_labels, alpha=.5):
    """Weight the loss values for original and adversarial examples
    according to hyperparameter alpha. https://arxiv.org/abs/1412.6572
    finds that alpha=.5 works well if the number of original and 
    adversarial examples is equal.
    """
    loss = alpha * F.nll_loss(og_output, og_labels) + (1 - alpha) * F.nll_loss(adv_output, adv_labels)
    return loss

### Define the train and test functions

In [7]:
# utility containers to record how training process goes, used for later visulaztions
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
adv_robustness = []

In [8]:
def train(epoch, adv=True):
    net.train()
    num_per_samp = 1
    adv_correct = 0

    for batch_idx, (og_samples, og_labels) in enumerate(train_loader):
        og_samples, og_labels = og_samples.to(0), og_labels.to(0)
        optimizer.zero_grad()
        
        og_output = net(og_samples)
        if adv:
            adv_samples, adv_labels = generate_adversarial_examples(og_samples, og_labels, num_per_samp=num_per_samp)
            adv_output = net(adv_samples)
#             loss = my_loss(og_output, og_labels, adv_output, adv_labels, alpha=.25)
            
            # compute accuracy on adversarial examples
            adv_preds = adv_output.data.max(1, keepdim=True)[1]
            adv_correct += adv_preds.eq(adv_labels.data.view_as(adv_preds)).sum().item()
            
            output = torch.cat([adv_output, og_output], 0)
            labels = torch.cat([adv_labels, og_labels], 0)

            loss = F.nll_loss(output, labels)
        else:
            loss = F.nll_loss(og_output, og_labels)
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(og_samples), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
    print(f'Training set adversarial robustness, i.e. percent correct classification on adversarial\nexamples created from training data: \n\tAccuracy: {(adv_correct/len(train_loader.dataset))*100:.2f}%)')
    

In [9]:
def test(adv=True):
    net.eval()
    
    og_test_loss = 0
    adv_test_loss = 0
    og_correct = 0
    adv_correct = 0
    
    with torch.no_grad():
        for sample, label in test_loader:
            sample, label = sample.to(0), label.to(0)
            output = net(sample)
            # negative log-likelihood loss
            og_test_loss += F.nll_loss(output, label, reduction='sum').item()
            # output is a tensor, .data retrieves its data, max must return the index of the highest valued element
            pred = output.data.max(1, keepdim=True)[1]
            og_correct += pred.eq(label.data.view_as(pred)).sum()
            
    if adv:
        for og_samples, og_labels in test_loader:
            og_samples, og_labels = og_samples.to(0), og_labels.to(0)
            adv_samples, adv_labels = generate_adversarial_examples(og_samples, og_labels, num_per_samp=1)
            adv_output = net(adv_samples)
            # negative log-likelihood loss
            adv_test_loss += F.nll_loss(adv_output, adv_labels, reduction='sum').item()
            # output is a tensor, .data retrieves its data, max must return the index of the highest valued element
            pred = adv_output.data.max(1, keepdim=True)[1]
            adv_correct += pred.eq(label.data.view_as(pred)).sum()
                
    og_test_loss /= len(test_loader.dataset)
    test_losses.append(og_test_loss)
    
    print(f'Test set: \n\tAvg. loss: {og_test_loss:.4f}\n\tAccuracy: {og_correct}/{len(test_loader.dataset)} ({100. * og_correct / len(test_loader.dataset):.0f}%)')
    
    og_pcc = 100. * og_correct / len(test_loader.dataset)

    if adv:
        adv_pcc = 100. * adv_correct / len(test_loader.dataset)
        return og_pcc, adv_pcc
    return og_pcc, None

### Define the function to create the adversarial examples

In [10]:
def generate_adversarial_examples(og_samples, true_labels, num_per_samp=1):
    """Create num_per_samp adversarial examples for each sample in
    og_samples. Return the generated samples along with corresponding 
    adv_labels, a tensor containing the adversarial examples' labels.
    """
    adversary = LinfPGDAttack(
        net, loss_fn=F.nll_loss, eps=0.15, nb_iter=40, eps_iter=0.01, 
        rand_init=True, clip_min=0.0, clip_max=1.0, targeted=False)
    adv_untargeted = []
    for i in range(num_per_samp):
        adv_untargeted.append(adversary.perturb(og_samples, true_labels))
    adv_untargeted = torch.cat(adv_untargeted, 0)
    adv_labels = torch.cat([true_labels]*num_per_samp, 0)
    return adv_untargeted, adv_labels

### Instantiate the network and train it

In [11]:
# # train the model WITHOUT adversarial examples
# net = Net()
# net.cuda()
# optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

# for epoch in range(1, n_epochs + 1):
#     train(epoch, adv=False)
#     og_pcc, adv_pcc = test(adv=True)
#     print(f'Test set:\n\tog_pcc = {og_pcc}%\n\tadv_pcc = {adv_pcc}%')
    
# train second version of model WITH adversarial examples
net = Net()
net.cuda()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
    
for epoch in range(1, n_epochs + 1):
    train(epoch, adv=True)
    og_pcc, adv_pcc = test(adv=True)
    print(f'Test set:\n\tOriginal data = {og_pcc}%\n\tAdversarial examples = {adv_pcc}%\n')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312384


KeyboardInterrupt: 

### Visualize the training / testing results

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
fig

In [ ]:
#torch.save(net.state_dict(), 'models/model1')

### Test robustness to adversarial examples created from training examples

In [ ]:
def test_robustness(num_per_samp):
    correct_preds = 0
    total_samps = 0
    for og_samples, og_labels in train_loader:
        og_samples, og_labels = og_samples.to(0), og_labels.to(0)
        adv_samples, adv_labels = generate_adversarial_examples(og_samples, og_labels, num_per_samp=num_per_samp)
        adv_output = net(adv_samples)
        preds = adv_output.data.max(1)[1]
        correct_preds += preds.eq(adv_labels.data.view_as(preds)).sum().item()
        total_samps += adv_samples.shape[0]
    return correct_preds/total_samps

In [ ]:
net = Net()
net.load_state_dict(torch.load('models/model1'))

In [ ]:
corr, tot = test_robustness(1)

In [ ]:
corr.item()/tot